<a href="https://colab.research.google.com/github/juliayul1a/SF_test/blob/main/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Summary:**
We have been given a PDF with information about an insurance applicant.
We are tasked with leveraging OpenAI and Retrieval-Augmented Generation (RAG) to review the application and give an informed recommendation as to whether or not a company should provide insurance to this person.


# Assumptions:


*   The provided application is for life insurance.
*   While document interpretation should not be overfit to this particular example, it is likely that future applications will follow similar patterns.
*   This is whole life insurance and the **premiums do not change**. Premiums and other pricing considerations could be easily modified in future versions.
*   Approval should be granted when the company can be reasonably sure that they will not lose money on the policy.
*   The company will not lose money if the combination of applicant payments + modest interest returns on the cumulative payments is greater than the payout amount of the policy after n_years.
*   If accepted, the applicant will pay for the policy for the lesser of: their remaining lifetime or 100 years. Future versions apply additional predicted information leveraging attrition/churn modeling.
*   For the purposes of this exercise, the total amount paid by the applicant is equal to n_years * annual_premium, though future versions could be adapted to respond to different payment plans.


# Approach:


1.   Collect basic demographic information about the individual (age, sex)
2.   Parse the document for additional information about the individual's health and habits in order to determine an appropriate risk class
3.   Using this information and guesstimated weights, calculate the risk score and assign a risk class of premium, standard, or sub-standard to the individual
4.   With the individual's age, sex and risk class, leverage Pandas agents to lookup pre-loaded and processed data to answer the following:

*   **Question 1**: Is the lifetime premium + investment outcome likely to yield an amount ≥ $250,000?

>*   **If no:** The company should reject the applicant.

>*   **If yes:** The company should find the greatest amount of insurance for which the applicant would be eligible.

*   **Question 2**:
For eligible applicants, what is the return the company will see at the highest eligible insurance tier?

# Outcome:
Provide the user with a summary of eligibility and final recommendations.


#Assets and Artifacts:

*   **[Billing codes](https://www.surgery.northwestern.edu/docs/cpt-codes.pdf):** Accessed via web 02.27.2024. Unmodified. Referenced via RAG.

*   **[Code Costs](https://www.cms.gov/research-statistics-data-and-systems/statistics-trends-and-reports/medicarefeeforsvcpartsab/downloads/level1charg13.pdf):** Accessed via web 02.27.2024. Slight modification and augmentation. Ultimately not leveraged in final solution.

*   **[Actuarial tables](https://www.ssa.gov/oact/STATS/table4c6.html):** Accessed via web 02.27.2024. Processing and augmentation applied. Referenced via Pandas agent.

*   **[Sample Policy Costs](https://www.policygenius.com/life-insurance/):** Accessed via web 02.27.2024. Heavily augmented via interpolation and extrapolation for use as sample data. Referenced via Pandas agent.

*   **[Sample Application]("/content/fake-aps.pdf"):** Provided as part of challenge. Unmodified. Referenced via RAG.



# Setup

**Install, load and define packages, UDFs and keys**

In [179]:
#All packages and imports for readability
## See also: requirements.txt

!pip install langchain openai weaviate-client PyMuPDF langchain_text_splitters tabula-py langchain_experimental tiktoken --quiet

import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import datetime
import tabula
import pandas as pd
from langchain.llms import OpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
import re
import textwrap

# UDFs:

def binary_var(i):
  if i == 'no':
    return 0
  else:
    return 1

def date_var(i):
  date_str = i
  format_str = "%m/%d/%Y"
  i = datetime.datetime.strptime(date_str, format_str)
  return i

## Keys:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openAPIKey2')
import os
os.environ['OPENAI_API_KEY'] = ai_key


**Clone Git and Get Files**

In [180]:
### Uncomment and clone as needed to load files & directory ###
# !git clone https://github.com/juliayul1a/SF_test.git

In [181]:
application_path = "/content/SF_test/sixfold_assets/fake-aps.pdf"
nu_codes_path = "https://www.surgery.northwestern.edu/docs/cpt-codes.pdf"
actuary_data_path = "/content/SF_test/sixfold_assets/actuarial_table.pdf"
female_cost_path = "/content/SF_test/sixfold_assets/female_costs.csv"
male_cost_path = "/content/SF_test/sixfold_assets/male_costs.csv"

# Load Application

**Primary tasks:**


*   Loads and processes the application for review
*   Finds critical information for the decision making process
*   Performs a general review and summary of the application


**Returns:**
Age, gender, risk, medical code info, etc.

In [182]:
##Load the application

loader = PyMuPDFLoader(application_path)
documents = loader.load()

Text splitting was not ultimately selected but is included below for process clarity

In [183]:
# ## We are leveraging a fairly large model so we have a generous window size. That said, having a chunk overlap of >25 caused some confusion with dates. A larger chunk and smaller overlap yielded the best results.
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
# chunks = text_splitter.split_documents(documents)

In [184]:
## Leveraging header split for better interpretability by the LLM. Previous iterations included chunks which led to confusion around page splits.

markdown_document = str(documents)

headers_to_split_on = [
    ("Date: ", "Header 1"),
    ("Date: ", "Header 2"),
    ("Date: ", "Header 3"),
    ("Date: ", "Header 4"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [185]:
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = md_header_splits,
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text = False
)

embedded weaviate is already listening on port 8079


In [186]:
retriever = vectorstore.as_retriever()

In [187]:
## Establishing prompt to promote clarity of answers

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
When answering, please provide only the answer and no additional text or information.
Here's an example: for the question, 'what is the applicant's name?' You should answer 'John Doe' instead of 'the applicant is named John Doe'
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nWhen answering, please provide only the answer and no additional text or information.\nHere's an example: for the question, 'what is the applicant's name?' You should answer 'John Doe' instead of 'the applicant is named John Doe'\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


In [188]:
## Invoking OpenAI gpt 3.5-turbo 0125 (also tried an earlier gpt) with temperature set to 0 in another effort to keep answers repeatable and factual.
llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125", temperature=0)


## Establishing rag_chain to answer our question leveraging our processed documents
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What is the applicant name?"
rag_chain.invoke(query)

'Sarah Miller'

In [189]:
## Gathering the critical information about our applicant in order to make our decision on approval. Leveraging several established variables as helpers for the queries to provide additional context.

#QuantQs:
age_query = "What is the applicant's age?"
age = rag_chain.invoke(age_query)

#QualQs:
name_query = "what is the applicant's name?"
gender_query = "What is the applicant's gender? Please answer male or female."
smoker_query = "Is there any indication that the applicant is a smoker? Please answer yes or no."
driving_query = "Is there any indication that the applicant has a poor driving record or DUIs? Please answer yes or no."

#Qual - StringQs:
applicant_name = str(rag_chain.invoke(name_query)).lower()
gender = str(rag_chain.invoke(gender_query)).lower()

#Qual - BinaryQs:
smoker = binary_var(str(rag_chain.invoke(smoker_query)).lower())
driving = binary_var(str(rag_chain.invoke(driving_query)).lower())


# #DateQs:
visit_dates_query = f"looking through all of the provided documents, what are all of the dates that {applicant_name} saw a doctor?"
first_visit_query = f"looking through all of the provided documents, what was the first date that {applicant_name} saw a doctor?"
last_visit_query = f"looking through all of the provided documents, what was the last date that {applicant_name} saw a doctor?"

visit_dates = rag_chain.invoke(visit_dates_query).split(",")
first_visit = date_var(rag_chain.invoke(first_visit_query))
last_visit = date_var(rag_chain.invoke(last_visit_query))

## Codes:
codes_by_date_query = f"what are the billing codes associated with each date in {visit_dates}? Please return the date and its corresponding code."
most_recent_code_query = f"what was the billing code for {applicant_name}'s last visit on {last_visit}?"

codes_by_date = rag_chain.invoke(codes_by_date_query)
most_recent_code = rag_chain.invoke(most_recent_code_query)

Summarization task:

In [190]:
## Establishing prompt assigning medical authority and requesting summary

sum_template = """You are medical professional who is being asked to provide a generalized summary on the health of an applicant.
This summary will be used as part of the decision making process to determine the applicant's eligibility for life insurance.
Based on your medical expertise and the information found in the application, please provide a summary and include any relevant details that you think would help inform the decision whether or not to approve the application.
Question: {question}
Context: {context}
Answer:
"""
sum_prompt = ChatPromptTemplate.from_template(sum_template)

print(sum_prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are medical professional who is being asked to provide a generalized summary on the health of an applicant.\nThis summary will be used as part of the decision making process to determine the applicant's eligibility for life insurance. \nBased on your medical expertise and the information found in the application, please provide a summary and include any relevant details that you think would help inform the decision whether or not to approve the application.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))]


In [191]:
## Invoking OpenAI gpt 3.5-turbo 0125 with higher temperature to allow for more creativity. Keeping other things (retriever, etc.) the same.
sum_llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125", temperature=1)


## Establishing rag_chain to answer our question leveraging our processed documents
sum_rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | sum_prompt
    | sum_llm
    | StrOutputParser()
)

query = f"What are your findings for {applicant_name}? Please include any relevant details such as the date of their last visit ({last_visit}), whether they are a smoker({smoker}) or a reckless driver ({driving}) and the nature of their last visit ({most_recent_code})"
applicant_summary = sum_rag_chain.invoke(query)

In [192]:
print(applicant_summary)

Based on the information provided in the application for Sarah Miller, she appears to be a healthy individual with a history of annual wellness exams and adherence to preventative health screenings. Her most recent visit on June 2, 2020, indicated that she presented for an annual physical and was up to date on immunizations, cancer screening, and gynecologic care. She reported feeling well and had a normal comprehensive exam. 

There is no mention of smoking or reckless driving in the information provided. Overall, based on the history of annual exams and the absence of significant health concerns, it is likely that Sarah Miller would be considered a low-risk applicant for life insurance.


# Billing Code Review

**Primary tasks:**


*   Loads and processes billing code reference data
*   Ingests billing codes found in the application and evaluates risk


**Returns:**
Code flag assignment (0 = benign or preventative, 5 = imminent mortal danger) & sum of all codes.

In [193]:
##Loading in medical reference codes with big thanks to my dear Alma Mater
nu_loader = PyMuPDFLoader(nu_codes_path)
nu_docs = nu_loader.load()


In [194]:
##Given this is in more tabular form than the application, character splitter was a fine choice and yielded high interpretability

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
nu_chunks = text_splitter.split_documents(nu_docs)

In [195]:
openai_api_key = OPENAI_API_KEY

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

nu_vectorstore = Weaviate.from_documents(
    client = client,
    documents = nu_chunks,
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text = False
)

nu_retriever = nu_vectorstore.as_retriever()

Started /root/.cache/weaviate-embedded: process ID 36449


Quick parsing of codes for interpretability. Leveraging AI over pandas(etc) in case of differences in response

In [196]:
print(codes_by_date)

7/12/1999 - 99212
11/18/2005 - 99395
8/12/2015 - 99212
6/2/2020 - 99396


In [197]:
parsed_codes = rag_chain.invoke(f"what are the medical codes in {codes_by_date}? please return only the codes.")

In [198]:
print(parsed_codes)

99212
99395
99212
99396


In [199]:
saved_codes = parsed_codes

In [200]:
##Establishing new prompt asking the LLM to assign a risk factor to each code provided:
nu_template = """You are a medical coder well versed in interpreting CPT codes.

You have been tasked with **reviewing all of the codes you are asked to review assigning each a risk rating from 0-5.**
To complete this task, you have been provided with a document which is a full catalog of CPT codes and their descriptions.
If the CPT code is a benign or preventative care measure, please assign a rating of 0. If it is a sign of imminent danger, please assign a rating of 5.

To complete your task, you must provide an answer of code : rating for each code you are asked to review. You do not need to explain your reasoning for the rating.
Question: {question}
context = {context}
answer = """

nu_prompt = ChatPromptTemplate.from_template(nu_template)

print(nu_prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are a medical coder well versed in interpreting CPT codes.\n\nYou have been tasked with **reviewing all of the codes you are asked to review assigning each a risk rating from 0-5.**\nTo complete this task, you have been provided with a document which is a full catalog of CPT codes and their descriptions.\nIf the CPT code is a benign or preventative care measure, please assign a rating of 0. If it is a sign of imminent danger, please assign a rating of 5.\n\nTo complete your task, you must provide an answer of code : rating for each code you are asked to review. You do not need to explain your reasoning for the rating.\nQuestion: {question}\ncontext = {context}\nanswer = '))]


In [201]:
##Returning temperature to zero for this model to promote repeatability, consistency of answers

openai_api_key = OPENAI_API_KEY
nu_llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo", temperature=0)

nu_rag_chain = (
    {"context": nu_retriever,  "question": RunnablePassthrough()}
    | nu_prompt
    | nu_llm
    | StrOutputParser()
)

nu_query = f"what ratings would you assign these codes: {parsed_codes}?"
code_answer = nu_rag_chain.invoke(nu_query)
print(code_answer)


99212: 0
99395: 0
99212: 0
99396: 0


In [202]:
##Gathering an interpretation of the ratings
flagged_codes = int(rag_chain.invoke(f"what is the sum of the values assigned to each code in {code_answer}? Please provide only the number."))

In [203]:
flagged_codes

0

In [204]:
##Finding the last code in order to assign different weights based on the recency of visits

In [205]:
last_code = int(rag_chain.invoke(f"please reference {code_answer} and find the value after the colon for code {most_recent_code}. Please provide only the value and not the code in your answer."))

In [206]:
last_code

0

# Weights & Class Assignments

**Primary tasks:**


*   Loads and assigns weights to the critical variables for making the decision


**Returns:**
Risk assessment class (risk_class) and score (risk_score)

In [207]:
cur_year = datetime.datetime.today().year

### Assign Weights:

smoker_weight = -2
driving_weight = 0
if driving == 0:
  driving_weight = 0
else:
  driving_weight = -0.5
flagged_code_weight = -0.5
year_dif = cur_year-last_visit.year
# medical_recency_weight = year_dif
medical_recency_weight = 0
if year_dif == 0:
  medical_recency_weight += 1
else:
  medical_recency_weight += -0.5 * (year_dif - 1)
last_code_weight = 0
if last_code == 0:
  last_code += 2
  last_code_weight += 1
else:
  last_code_weight += -1

# Calculate Risk

risk_calc = (
    smoker * smoker_weight
    + driving * driving_weight
    + medical_recency_weight
    + last_code * last_code_weight
    + flagged_codes * flagged_code_weight
)

## Assign Risk Class
risk_class = ''
if risk_calc >= 1.5:
  risk_class += 'premium'
elif -1.5 <= risk_calc <= 1.4:
  risk_class += 'standard'
else:
  risk_class += 'sub-standard'

In [208]:
risk_calc

0.5

# Actuary Data

**Primary tasks:**


*   Identify **n_years** of possible policy tenure by gender

**Key Inputs:**


*   **age**
*   **gender** (binary for the purposes of this exercise)

**Data Leveraged:**


*   **[Actuarial tables](https://www.ssa.gov/oact/STATS/table4c6.html):** Accessed via web 02.27.2024. Processing and augmentation applied. Referenced via Pandas agent.

**Returns:**
DFs: [actuary_female, actuary_male]

**Ingest Actuarial Table Data**

In [209]:
## Establish PDF Location
pdf_path = actuary_data_path

## Process page 1 (slightly different processing steps than 2-3 & 4)
page1 = pd.DataFrame(tabula.read_pdf(pdf_path)[0])

## Define function for pages 2 & 3 which both require the same processing steps
def extract_page_data(pdf_path, page_number):
    df = pd.DataFrame(
        tabula.read_pdf(pdf_path, stream=True, pages=str(page_number), pandas_options={'header': None})[0]
    )
    df = df.iloc[4:]
    df.reset_index(drop=True, inplace=True)
    return df

## Apply function to process pages 2 & 3
page2 = extract_page_data(pdf_path, 2)
page3 = extract_page_data(pdf_path, 3)

##Define coordinates of table for page 4 (gathered using Mac Preview + Crop– see StackOF discussion here: http://tinyurl.com/mphkt9pb)
top = 13.77
left = 31.64
bottom = top + 524.28
right = left + 535.74

## Process page 4 (slightly different processing steps than 1 & 2-3 given need to define area)
page4 = pd.DataFrame(tabula.read_pdf(pdf_path, stream=True, pages ="4", area=[top, left, bottom, right], pandas_options={'header':None})[0])
page4 = page4.iloc[4:]
page4.reset_index(drop=True, inplace=True)


In [210]:
##Clean up page 1 & assign headers
act_df = page1
actuarial = act_df.iloc[3:]
# actuarial.head()
actuarial.columns = ['age', 'male_p_death', 'male_num_lives', 'male_life_expt',
              'female_p_death', 'female_num_lives', 'female_life_expt']
actuarial.reset_index(drop=True, inplace=True)

# Display the modified dataframe
actuarial.head(110)

,age,male_p_death,male_num_lives,male_life_expt,female_p_death,female_num_lives,female_life_expt
0,0,0.005837,"100,000",74.12,0.004907,"100,000",79.78
1,1,0.000410,"99,416",73.55,0.000316,"99,509",79.17
2,2,0.000254,"99,376",72.58,0.000196,"99,478",78.19
3,3,0.000207,"99,350",71.60,0.000160,"99,458",77.21
4,4,0.000167,"99,330",70.62,0.000129,"99,442",76.22
5,5,0.000141,"99,313",69.63,0.000109,"99,430",75.23
6,6,0.000123,"99,299",68.64,0.000100,"99,419",74.24
7,7,0.000113,"99,287",67.65,0.000096,"99,409",73.25
8,8,0.000108,"99,276",66.65,0.000092,"99,399",72.25
9,9,0.000114,"99,265",65.66,0.000089,"99,390",71.26


In [211]:
## Shorten names for easier concatenation readability
p1 = actuarial
p2 = page2
p3 = page3
p4 = page4

## Assign p1 columns to each additional page

dataframes = [p2, p3, p4]

for df in dataframes:
    df.columns = p1.columns


## Concatenate pages into one table (120 x 7)

actuary = pd.concat([p1, p2, p3, p4], axis=0,ignore_index=True)
actuary.apply(pd.to_numeric, errors='coerce')
actuary.head()
# cols = ['col1', 'col2', 'col3']
# data[cols] = data[cols].apply(pd.to_numeric, errors='coerce', axis=1)

,age,male_p_death,male_num_lives,male_life_expt,female_p_death,female_num_lives,female_life_expt
0,0,0.005837,"100,000",74.12,0.004907,"100,000",79.78
1,1,0.000410,"99,416",73.55,0.000316,"99,509",79.17
2,2,0.000254,"99,376",72.58,0.000196,"99,478",78.19
3,3,0.000207,"99,350",71.60,0.000160,"99,458",77.21
4,4,0.000167,"99,330",70.62,0.000129,"99,442",76.22


In [212]:
#sanity check:
len(actuary)

120

**Give additional calculations and clarity to Actuary Table**

In [213]:
# Initialize p_survive columns for male and female
actuary.insert(loc= 2, column='male_p_survive', value=[0 for i in range(actuary.shape[0])])
actuary.insert(loc= 6, column='female_p_survive', value=[0 for i in range(actuary.shape[0])])

# Set both p_death columns to numeric for calculations
actuary['male_p_death'] = pd.to_numeric(actuary.male_p_death)
actuary['female_p_death'] = pd.to_numeric(actuary.female_p_death)


# Apply calc 1-p_death to each row in actuary for male and female to calculate p_survive
for i in range(len(actuary)):
    p_death = actuary.loc[i, 'male_p_death']
    actuary.loc[i, 'male_p_survive'] = 1 - p_death

for i in range(len(actuary)):
    p_death = actuary.loc[i, 'female_p_death']
    actuary.loc[i, 'female_p_survive'] = 1 - p_death

actuary.head()


,age,male_p_death,male_p_survive,male_num_lives,male_life_expt,female_p_death,female_p_survive,female_num_lives,female_life_expt
0,0,0.005837,0.994163,"100,000",74.12,0.004907,0.995093,"100,000",79.78
1,1,0.000410,0.999590,"99,416",73.55,0.000316,0.999684,"99,509",79.17
2,2,0.000254,0.999746,"99,376",72.58,0.000196,0.999804,"99,478",78.19
3,3,0.000207,0.999793,"99,350",71.60,0.000160,0.999840,"99,458",77.21
4,4,0.000167,0.999833,"99,330",70.62,0.000129,0.999871,"99,442",76.22


In [214]:
actuary_male = actuary[['age', 'male_p_death', 'male_p_survive', 'male_num_lives', 'male_life_expt']].copy()
actuary_female = actuary[['age', 'female_p_death', 'female_p_survive', 'female_num_lives', 'female_life_expt']].copy()


In [215]:
## Drop gender flags from column names for both dfs & write out words for clarity and readability

actuary_male.columns = ['age', 'probability_death', 'probability_survive', 'num_lives', 'life_expectancy']
actuary_female.columns = actuary_male.columns

#Confirm change with 2nd table:
actuary_female.head()

,age,probability_death,probability_survive,num_lives,life_expectancy
0,0,0.004907,0.995093,"100,000",79.78
1,1,0.000316,0.999684,"99,509",79.17
2,2,0.000196,0.999804,"99,478",78.19
3,3,0.000160,0.999840,"99,458",77.21
4,4,0.000129,0.999871,"99,442",76.22


In [216]:
##Quick sanity check
print(len(actuary_female), len(actuary_male))

120 120


**END ACTUARY BLOCK**

DFs = [actuary_female, actuary_male]

# Cost Data by Gender and Age


**Primary tasks:**


*   Serve as reference for annual premium costs based on the applicant's age, sex and risk class.

**Key Inputs:**


*   **age**
*   **gender** (Binary for the purposes of this exercise)
*   **risk_class**

**Data & Other Assets:**


*   Two instances of sample cost data for whole life insurance policies

**[Sample Policy Costs](https://www.policygenius.com/life-insurance/)**: Accessed via web 02.27.2024. Heavily augmented via interpolation and extrapolation for use as sample data. Referenced via Pandas agent.

**Returns:**

DFs: [female_costs, male_costs]

In [217]:
male_costs = pd.read_csv(male_cost_path)
female_costs = pd.read_csv(female_cost_path)
female_costs.head()


,Age,Gender,250_monthly_cost,250_annual_cost,500_monthly_cost,500_annual_cost,1m_monthly_cost,1m_annual_cost
0,20,Female,$149.00,"$1,788.00",$293.00,"$3,516.00",$572.00,"$6,864.00"
1,21,Female,$155.40,"$1,864.80",$305.80,"$3,669.60",$597.50,"$7,170.00"
2,22,Female,$161.80,"$1,941.60",$318.60,"$3,823.20",$623.00,"$7,476.00"
3,23,Female,$168.20,"$2,018.40",$331.40,"$3,976.80",$648.50,"$7,782.00"
4,24,Female,$174.60,"$2,095.20",$344.20,"$4,130.40",$674.00,"$8,088.00"


# Pandas Agents and/or maybe just Pandas

**The pandas agents are under "experimental" for a reason and are too mercurial for this task. They have been relieved of their duties for now.**

**Actuary Agent**

For simplicty, identify the gender of the applicant and assign a table accordingly.

Pass that table to the agent.

Ask the agent to query the table for the desired information.

**Returns:** num_years (life_expectancy given age)

In [218]:
# Assign either male or female data to actuary_table based on applicant context
actuary_table = []
if gender == 'female':
  actuary_table = actuary_female
else:
  actuary_table = actuary_male

In [219]:
# # Create actuary_agent
# actuary_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), actuary_table, verbose=False)

In [220]:
# ### Warm up the agent
# answer1 = actuary_agent.run("how many rows are in the table? Please answer with just a number")
# answer2 = actuary_agent.run("how many columns are in the table? Please answer with just a number")
# answer3 = actuary_agent.run("What are the different columns in the df?")
# answer4 = actuary_agent.run("What is the age range found in the table?")
# print(answer1, answer2, answer3, answer4)

In [221]:
# # Identify life_expectancy given age and set to int
# life_expectancy = actuary_agent.run(f"Find age {age} in the Age column and return life_expectancy. Please reply with a number.")
# print(life_expectancy)

In [222]:
actuary_table.head(5)

,age,probability_death,probability_survive,num_lives,life_expectancy
0,0,0.004907,0.995093,"100,000",79.78
1,1,0.000316,0.999684,"99,509",79.17
2,2,0.000196,0.999804,"99,478",78.19
3,3,0.000160,0.999840,"99,458",77.21
4,4,0.000129,0.999871,"99,442",76.22


In [223]:
life_expectancy = actuary_table.loc[actuary_table['age'] == age, 'life_expectancy'].values[0]
life_expectancy

'42.31'

**The pandas agents are under "experimental" for a reason and are too mercurial for this task. They have been relieved of their duties for now.**

**Cost Agent**

For simplicty, identify the gender of the applicant and assign a table accordingly. This will prevent us from surpassing our token window.

Pass that table to the agent.

Ask the agent to query the table for the desired information.

**Returns:** annual_cost for a 250k, 500k and 1M policy

In [224]:
# Assign either male or female data to cost_table based on applicant context

cost_table = []
if gender == 'female':
  cost_table = female_costs
else:
  cost_table = male_costs

In [225]:
cost_table.head(5)

,Age,Gender,250_monthly_cost,250_annual_cost,500_monthly_cost,500_annual_cost,1m_monthly_cost,1m_annual_cost
0,20,Female,$149.00,"$1,788.00",$293.00,"$3,516.00",$572.00,"$6,864.00"
1,21,Female,$155.40,"$1,864.80",$305.80,"$3,669.60",$597.50,"$7,170.00"
2,22,Female,$161.80,"$1,941.60",$318.60,"$3,823.20",$623.00,"$7,476.00"
3,23,Female,$168.20,"$2,018.40",$331.40,"$3,976.80",$648.50,"$7,782.00"
4,24,Female,$174.60,"$2,095.20",$344.20,"$4,130.40",$674.00,"$8,088.00"


In [226]:
# age = int(age)
# # Create cost_agent
# cost_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), cost_table, verbose=False)

In [227]:
# # Given age, identify the annual premium for $250, $500 and $1M policies
# annual_premium_250 = actuary_agent.run(f"what is 250_annual_cost for someone who is {age} years old?")
# annual_premium_250

In [228]:
## Cost agent was unceremoniously fired.

In [229]:
# Filter cost_table by age
cost_filtered = cost_table[cost_table['Age'] == int(age)]


# Assigning to unique variables
annual_premium_250 = re.sub(r'[^\d.]','', cost_filtered["250_annual_cost"].values[0])
annual_premium_500 = re.sub(r'[^\d.]','', cost_filtered["500_annual_cost"].values[0])
annual_premium_1M = re.sub(r'[^\d.]','', cost_filtered["1m_annual_cost"].values[0])




In [230]:
def risk_adjustment(i):
  if risk_class == 'premium':
    i = i * .95
  elif risk_class == 'sub-standard':
    i = i * 1.05
  else:
    i = i
  return i

In [231]:
annual_premium_250 = risk_adjustment(annual_premium_250)
annual_premium_500 = risk_adjustment(annual_premium_500)
annual_premium_1M = risk_adjustment(annual_premium_1M)

# Key Info

**Gathering up all of the key information to return the final decision**

In [232]:
key_info = (age, gender, risk_class, num_years)

In [233]:
key_info

('39',
 'female',
 'standard',
 'The life expectancy for someone who is 39 years old cannot be found in the given dataframe.')

In [234]:
policy_amounts = [250000, 500000, 1000000]


# Should Approve?

**Review the key information and look to see if, given the life expectancy of the individual + expected returns, the policy is a good investment for the company** (i.e. they aren't likely to lose money on it)

Future iterations could include churn prediction and other methods for shoring up these estimates.

In [239]:
num_years = int(float(life_expectancy))
annual_premium = float(annual_premium_250)
annual_premium = int(annual_premium)
lifetime_premium = num_years * annual_premium
print(lifetime_premium)
### Remember to cap out years at 100

153132


In [240]:
##Assume modest interest earned on policy premiums
def calculate_interest(initial_investment, years, average_apy):

  # Convert APY to decimal format
  apy_decimal = average_apy / 100

  # Calculate the total amount invested
  total_invested = initial_investment * (years + 1)

  # Calculate the total interest earned
  total_interest = total_invested * apy_decimal * years

  return total_interest

# Example usage
initial_investment = annual_premium
years = num_years
average_apy = 2.1

total_interest = calculate_interest(initial_investment, years, average_apy)

print(f"Total interest earned: ${total_interest:.2f}")


Total interest earned: $138278.20


In [241]:
total_dollars = lifetime_premium + total_interest

In [242]:
lowest_threshold = policy_amounts[0]
lowest_threshold = int(lowest_threshold)

In [243]:
should_insure = 0
if lowest_threshold > total_dollars:
  should_insure += 0
else:
  should_insure += 1

In [244]:
should_insure

1

In [245]:
decision = ''
if should_insure == 1:
  decision = "should be insured"
else:
  decision = "should not be approved for insurance at this time."

# If approved, at what amount?

**If the applicant is approved, review the key information to see the highest amount at which the applicant should be insured.**

Future iterations could include options for different insurance policies (i.e. term, etc.)

In [248]:
## Find the amounts for 250, 500 and 1M:


##250
annual_premium_250 = float(annual_premium_250)
annual_premium_250 = int(annual_premium_250)
lifetime_premium_250 = num_years * annual_premium_250

initial_investment = annual_premium_250
years = num_years
average_apy = 2.1

total_interest_250 = calculate_interest(initial_investment, years, average_apy)

total_dollars_250 = total_interest_250 + lifetime_premium_250

##500
annual_premium_500 = float(annual_premium_500)
annual_premium_500 = int(annual_premium_500)
lifetime_premium_500 = num_years * annual_premium_500

initial_investment = annual_premium_500
years = num_years
average_apy = 2.1

total_interest_500 = calculate_interest(initial_investment, years, average_apy)

total_dollars_500 = total_interest_500 + lifetime_premium_500

##1M
annual_premium_1M = float(annual_premium_1M)
annual_premium_1M = int(annual_premium_1M)
lifetime_premium_1M = num_years * annual_premium_1M

initial_investment = annual_premium_1M
years = num_years
average_apy = 2.1

total_interest_1M = calculate_interest(initial_investment, years, average_apy)

total_dollars_1M = total_interest_1M + lifetime_premium_1M

In [249]:
approved_amount = 0
##Check 1M First
if total_dollars_1M >= policy_amounts[2]:
  approved_amount += policy_amounts[2]
  annual_premium = annual_premium_1M
  lifetime_premium = lifetime_premium_1M
  total_interest = total_interest_1M
  total_dollars = total_dollars_1M
##Check 500k Second
elif total_dollars_500 >= policy_amounts[1]:
  approved_amount += policy_amounts[1]
  annual_premium = annual_premium_500
  lifetime_premium = lifetime_premium_500
  total_interest = total_interest_500
  total_dollars = total_dollars_500
else:
  approved_amount += policy_amounts[0]
  annual_premium = annual_premium_250
  lifetime_premium = lifetime_premium_250
  total_interest = total_interest_250
  total_dollars = total_dollars_250


total_return = approved_amount - total_dollars

In [250]:
approved_amount

1000000

# Conclusion

In [254]:
## Pulling together the final decision and business case:
conclusion = ''

if should_insure == 1:
  conclusion = (f"{applicant_name}, a {age} year old {gender}, {decision} up to an amount of ${approved_amount} at {risk_class} rates. The annual premium for this policy is expected to be around ${annual_premium} for a term of approximately {num_years} years. If properly invested, the premium for this policy is projected to return ${total_interest} in interest during the term period based on a rate of {average_apy}% APY.")
else:
  conclusion = f"{applicant_name}, a {age} year old {gender}, {decision}"

final_conclusion = (conclusion + '\n'+'\n' + applicant_summary)

print(textwrap.fill(final_conclusion, width=250))

sarah miller, a 39 year old female, should be insured up to an amount of $1000000 at standard rates. The annual premium for this policy is expected to be around $14265 for a term of approximately 42 years. If properly invested, the premium for this
policy is projected to return $541014.39 in interest during the term period based on a rate of 2.1% APY.  Based on the information provided in the application for Sarah Miller, she appears to be a healthy individual with a history of annual wellness
exams and adherence to preventative health screenings. Her most recent visit on June 2, 2020, indicated that she presented for an annual physical and was up to date on immunizations, cancer screening, and gynecologic care. She reported feeling well
and had a normal comprehensive exam.   There is no mention of smoking or reckless driving in the information provided. Overall, based on the history of annual exams and the absence of significant health concerns, it is likely that Sarah Miller would
be

Set up git requirements text

In [255]:
## Gather Packages for requirements.txt for github
!pip freeze > requirements.txt
